In [1]:
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.18


In [2]:
# Imports
import re
import nltk
import string
import numpy as np
import pandas as pd
from os import getcwd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

nltk      : 3.8.1
numpy     : 1.26.3
matplotlib: 3.8.2
pandas    : 2.1.4
re        : 2.2.1



## Extração de Dados

In [6]:
# Download de amostras de dados
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/frnepom/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [7]:
# Download as stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/frnepom/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# Verificar a pasta corrente
getcwd()

'/Users/frnepom/Documents/Courses/Data_Science_Academy/Formacao_Analise_Estatistica/Matematica_Para_DS/CAP05'

In [9]:
# Importa o Corpus (dados)
from nltk.corpus import twitter_samples

In [10]:
type(twitter_samples)

nltk.corpus.util.LazyCorpusLoader

## Preparação dos Dados

O objeto `twitter_samples` contém subconjuntos de 5 mil tweets positivos, 5 mil tweets negativos e o conjunto completo de 10.000 tweets.

Vamos trabalhar com as duas amostras de 5 mil tweets cada uma.

In [11]:
tweets_positivos = twitter_samples.strings('positive_tweets.json')

In [12]:
len(tweets_positivos)

5000

In [13]:
# Visualizamos alguns tweets positivos
tweets_positivos[2:8]

['@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days',
 '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM',
 "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI",
 '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.']

In [14]:
# Carregamos tweets negativos
tweets_negativos = twitter_samples.strings('negative_tweets.json')

In [15]:
len(tweets_negativos)

5000

In [16]:
# Visualizamos alguns tweets negativos
tweets_negativos[2:8]

['@Hegelbon That heart sliding into the waste basket. :(',
 '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
 'Dang starting next week I have "work" :(',
 "oh god, my babies' faces :( https://t.co/9fcwGvaki0",
 '@RileyMcDonough make me smile :((',
 '@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln']

In [17]:
type(tweets_negativos)

list

## Divisão em Treino e Teste

Vamos dividir os dados com uma proporção 80/20 (treino/teste) garantindo a mesma proporção de tweets positivos e negativos.

In [18]:
# Split dos dados de tweets positivos - 1000 tweets para dados de teste e 4000 tweets para dados de treino
tweets_positivos_teste = tweets_positivos[4000:]
tweets_positivos_treino = tweets_positivos[:4000]

In [19]:
# Split dos dados de tweets negativos - 1000 tweets para dados de teste e 4000 tweets para dados de treino
tweets_negativos_teste = tweets_negativos[4000:]
tweets_negativos_treino = tweets_negativos[:4000]

In [20]:
# Concatena os dados de treino
dados_treino_x = tweets_positivos_treino + tweets_negativos_treino

In [21]:
len(dados_treino_x)

8000

In [22]:
# Concatena os dados de teste
dados_teste_x = tweets_positivos_teste + tweets_negativos_teste

In [23]:
len(dados_teste_x)

2000

> Criamos então um vetor (array) numpy de rótulos positivos e negativos.

In [24]:
# Combinando labels positivos e negativos de treino
y_treino = np.append(np.ones((len(tweets_positivos_treino), 1)),
                     np.zeros((len(tweets_negativos_treino),1)), axis=0)

In [25]:
y_treino.shape

(8000, 1)

In [26]:
# Combinando labels positivos e negativos de teste
y_teste = np.append(np.ones((len(tweets_positivos_teste), 1)),
                    np.ones((len(tweets_positivos_teste), 1)), axis=0)

In [27]:
y_teste.shape

(2000, 1)

## Manipulação de Texto e Pré-Processamento dos Dados

Vamos criar uma função para processar o texto dos posts do Twitter.

In [31]:
def limpa_processa_tweet(tweet):
    
    # Stop words
    stopwords_english = stopwords.words('english')
    
    # Remove stock market tickers (símbolo $GE)
    tweet = re.sub(r'\$\w*', '', tweet)
    
    # Remove texto de retweet ("RT")
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # Remove hyperlinks
    tweet = re.sub(r'http?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # Remove hashtags
    tweet = re.sub(r'#', '', tweet)
    
    # Cria um tokenizador (separa frases em palavras)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    
    # Aplica o tokenizador
    tweet_tokens = tokenizer.tokenize(tweet)
    
    # Lista para os tweets limpos
    tweets_clean = []
    
    # Cria o objeto Stemmer
    stemmer = PorterStemmer()
    
    # Loop
    for word in tweet_tokens:
        
        # Removemos stop words e pontuação
        if(word not in stopwords_english and word not in string.punctuation):
            
            # Stemming (Extraindo os radicais das palavras)
            stem_word = stemmer.stem(word)
            
            # Tweets limpos
            tweets_clean.append(stem_word)
            
    return tweets_clean
    
    

> Vamos testar o processamento

In [32]:
print('Este é um exemplo de tweet positivo original: \n\n', dados_treino_x[0])

Este é um exemplo de tweet positivo original: 

 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [33]:
print('\nEste é um exemplo da versã processada do tweet: \n\n', limpa_processa_tweet(dados_treino_x[0]))


Este é um exemplo da versã processada do tweet: 

 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


> Vamos criar uma função para construir o dicionário de frequência das palavras.

In [35]:
# Função para criar o dicionário de frequência
def cria_freqs(tweets, ys):
    
    # tweets é a lista de tweets
    # ys é um array m x 1 com label de sentimento para cada tweet (0 ou 1)
    
    # Squeeze (remove uma das dimensões)
    yslist = np.squeeze(ys).tolist()
    
    # Dicionário de frequências
    freqs = {}
    
    # Loop para cada tweet
    for y, tweet in zip(yslist, tweets):
        
        # Loop para cada palavra
        for word in limpa_processa_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [36]:
dados_treino_x[0:5]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days']

In [37]:
y_treino[0:5]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [38]:
# Aplica a função aos dados de treino
freqs = cria_freqs(dados_treino_x, y_treino)

In [39]:
type(freqs)

dict

In [41]:
dict(list(freqs.items())[50:55])

{('long', 1.0): 27,
 ('hope', 1.0): 113,
 ('enjoy', 1.0): 57,
 ('happi', 1.0): 161,
 ('friday', 1.0): 91}

In [42]:
dict(list(freqs.items())[6995:7000])

{('ticket', 0.0): 9,
 ('codi', 0.0): 1,
 ('simpson', 0.0): 1,
 ('concert', 0.0): 9,
 ('singapor', 0.0): 3}

In [43]:
freqs

{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

## Modelagem Preditiva

Usaremos o algoritmo de Regressão Logística para classificação dos tweets em positivos ou negativos. 

Vamos construir cada etapa matemática desse algoritmo.

### Parte 1:  Matemática da Função Sigmóide

A função sigmóide é uma função de ativação comumente usada em redes neurais. A função sigmóide é definida como: 

$$ h(z) = \frac{1}{1+\exp^{-z}} $$

Ela tem a forma de uma curva S, como mostrado abaixo:

![title](imagens/sigmoid.png)

A função sigmóide tem a seguintes propriedades:

O valor de saída da função sigmóide está sempre entre 0 e 1. Isso torna a função útil para problemas de classificação binária, pois pode ser interpretada como a probabilidade de um determinado exemplo pertencer à classe positiva.

A função sigmóide é derivável em todos os pontos, o que a torna útil para o treinamento de redes neurais.

A função sigmóide tem um gradiente muito pequeno para valores de entrada muito grandes ou muito pequenos. Isso pode causar problemas durante o treinamento da rede neural, pois pode levar ao "estouro do gradiente", um problema em que o gradiente fica muito grande e a rede neural deixa de aprender de maneira eficiente.

Apesar desses problemas, a função sigmóide ainda é usada em alguns casos, especialmente em problemas de classificação binária. No entanto, outras funções de ativação, como a ReLU e a tangente hiperbólica, são mais comumente usadas em redes neurais profundas devido ao seu desempenho melhor.

Vamos implementar em Python. Referência:

https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html

In [44]:
# Função sigmóide
def sigmoide(z):
    
    # Calcula o sigmóide de z
    h = 1 / (1 + np.exp(-z))
    
    return h

In [45]:
# Testando a função
sigmoide(1)

0.7310585786300049

In [46]:
# Testando a função
if (sigmoide(1) == 0.7310585786300049):
    print('CORRECT!')
else:
    print('WRONG!')

CORRECT!


In [47]:
sigmoide(3.92)

0.9805449154318069

In [48]:
if (sigmoide(1) == 0.91):
    print('CORRECT!')
else:
    print('WRONG!')

WRONG!


### Parte 5: Combinando Todas as Operações e Criando o Algoritmo

* O número de iterações 'num_iters" é o número de vezes que você usará todo o conjunto de treinamento (número de passadas de treinamento).


* Para cada iteração, você calculará a função de custo usando todos os exemplos de treinamento (existem 'm' exemplos de treinamento) e para todos os recursos.


* Em vez de atualizar um único peso $\theta_i$ de cada vez, podemos atualizar todos os pesos no vetor de coluna:

$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$

* $\mathbf{\theta}$ tem dimensões (n+1, 1), onde 'n' é o número de recursos e há mais um elemento para o termo de viés $\theta_0$.


* Os 'logits', 'z', são calculados multiplicando a matriz de características 'x' pelo vetor de peso 'theta'. $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ tem dimensões (m, n+1) 
    * $\mathbf{\theta}$: tem dimensões (n+1, 1)
    * $\mathbf{z}$: tem dimensões (m, 1)


* A predição 'h', é calculada aplicando o sigmóide a cada elemento em 'z': $h(z) = sigmoid(z)$, e tem dimensões (m,1).


* A função de custo $J$ é calculada através do produto escalar dos vetores 'y' e 'log(h)'. Como 'y' e 'h' são vetores coluna (m,1), fazemos a transposta do vetor para a esquerda, de modo que a multiplicação da matriz de um vetor linha com o vetor coluna execute o produto escalar.

$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$


* A atualização de $\theta$ também é vetorizada. Como as dimensões de $\mathbf{x}$ são (m, n+1) e $\mathbf{h}$ e $\mathbf{y}$ são (m, 1), precisamos transpor o $ \mathbf{x}$ à esquerda para realizar a multiplicação de matrizes, o que resulta na resposta (n+1, 1) de que precisamos:

$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [49]:
def algo_reg_log(x, y, theta, alpha, n_iters):
    
    # Obter 'm', o número de linhas na matriz x
    m = x.shape[0]
    
    for i in range(0, n_iters):
        
        # Obter z, o produto escalar x e theta
        z = np.dot(x, theta)
        
        # Obter sigmoid de h
        h = sigmoide(z)
        
        # Calcula a funçã de custo
        # Note que podemos usar também np.array.transpose() ao invés de np.array.T
        # np.array.T apenas torna o código um pouco menos legível
        J = -1/m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))
        
        # Atualiza os pesos theta
        theta = theta - (alpha/m) * np.dot(x.T,(h-y))
        
    J = float(J)
    
    return J, theta

> Vamos testar a função

In [50]:
# Seed
np.random.seed(1)

In [51]:
# A entrada X é 10 x 3 com 1 para o termod de viés
dados_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)

In [52]:
# Labels Y são 10 x 1
dados_Y = (np.random.rand(10, 1) > 0.35).astype(float)

In [53]:
# Aplica a função
valor_J, valor_theta = algo_reg_log(dados_X, dados_Y, np.zeros((3,1)), 1e-8, 700)

/var/folders/42/fjzlf7bn6573xz4q1rvt4rpm0000gn/T/ipykernel_7059/2935699527.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [55]:
print(f'\nCusto (erro) após o treinamento é {valor_J:.8f}')
print(f'\nO vetor de pesos resultante é {[round(t, 8) for t in np.squeeze(valor_theta)]}')


Custo (erro) após o treinamento é 0.67094970

O vetor de pesos resultante é [4.1e-07, 0.00035658, 7.309e-05]


## Extração de Atributos

Com o algoritmo pronto vamos extrair os atributos dos dados e treinar um modelo.

* Dada uma lista de tweets, extraímos os recursos e armazenamos em um vetor. Vamos extrair dois recursos:

     * A primeira característica é o número de palavras positivas em um tweet.
     * A segunda característica é o número de palavras negativas em um tweet.

A função abaixo realiza as seguintes tarefas:

* Processa o tweet usando a função `limpa_processa_tweet` e salvamos a lista de palavras do tweet.


* Percorre cada palavra na lista de palavras processadas.


* Para cada palavra, verificamos o dicionário de frequências 'freqs' para a contagem quando essa palavra tiver um rótulo '1' positivo. Fazemos o mesmo para a contagem quando a palavra estiver associada ao rótulo negativo '0'. 

In [57]:
# Função para extração de atributos
def func_extract_features(tweet, freqs):
    
    # Aplica a função de limpeza e processamento
    word_l = limpa_processa_tweet(tweet)
    
    # Cria o vetro x de 3 elementos na forma 1 x 3
    x = np.zeros((1, 3))
    
    # O termo de bias será definido como 1
    x[0, 0] = 1
    
    # Loop pleas palavras
    for word in word_l:
        
        # Palavra de tweet positivo
        x[0, 1] += freqs.get((word, 1.0), 0)
        
        # Palavra de tweet negativo
        x[0, 2] += freqs.get((word, 0.0), 0)
        
    assert(x.shape == (1, 3))
    
    return x

## Treinamento do Modelo

A função abaixo realiza as seguintes tarefas:

* Processa o tweet usando a função `limpa_processa_tweet` e salvamos a lista de palavras do tweet.


* Percorre cada palavra na lista de palavras processadas.


* Para cada palavra, verificamos o dicionário de frequências 'freqs' para a contagem quando essa palavra tiver um rótulo '1' positivo. Fazemos o mesmo para a contagem quando a palavra estiver associada ao rótulo negativo '0'. 

In [58]:
# Criamos a matriz X
X = np.zeros((len(dados_treino_x), 3))

In [59]:
# Loop para preencher a matriz com os dados
for i in range(len(dados_treino_x)):
    X[i, :] = func_extract_features(dados_treino_x[i], freqs)

In [60]:
# Variável de saída (target)
Y = y_treino

In [61]:
# Hiperparâmetros

# Valor inicial da matriz de pesos
matriz_pesos = np.zeros((3,1))

# Taxa de aprendizado
taxa_aprendizado_alfa = 1e-9

# Número de iterações
num_iters = 1500

In [62]:
# Treinando do modelo
custo, pesos = algo_reg_log(X,Y, matriz_pesos, taxa_aprendizado_alfa, num_iters)

/var/folders/42/fjzlf7bn6573xz4q1rvt4rpm0000gn/T/ipykernel_7059/2935699527.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [63]:
print(f'O Custo (Erro) de Treinamento foi {custo:.8f}.')

O Custo (Erro) de Treinamento foi 0.24215478.


In [64]:
print(f'O Vetor de Pesos é {[round(t, 8) for t in np.squeeze(pesos)]}')

O Vetor de Pesos é [7e-08, 0.00052391, -0.00055517]


São 3 pesos pois temos 2 atributos de entrada e o bias.

## Previsões com o Modelo

É hora de fazer previsões com o modelo de regressão logística em alguma nova entrada de dados. O objetivo é prever se um tweet tem sentimento positivo ou negativo.

Vamos criar uma função para isso que executará as seguintes tarefas:

* Dado um tweet, processamos e extraímos os recursos (o que mesmo que foi feito nso dados de treino).
* Aplicamos os pesos aprendidos do modelo para obter os logits.
* Aplicamos a função sigmóide aos logits para obter a previsão (um valor entre 0 e 1).

Resumindo:

$$y_{pred} = sigmoide(\mathbf{x} \cdot \theta)$$

In [69]:
# Função para previsão
def func_previsao(tweet, freqs, theta):
    
    # Extrai os atributos
    x = func_extract_features(tweet, freqs)
    
    # Faz a previsão
    y_pred = sigmoide(np.dot(x, theta))
    
    return y_pred

In [70]:
# Vamos testar a função
for tweet in [':)', 
              ':(', 
              'I am happy', 
              'This course is great', 
              'I do not expect so much from my soccer team', 
              'It was a good book', 
              'I am not sure about the text']:
    print( '%s _> %f' % (tweet, func_previsao(tweet, freqs, pesos))) 

:) _> 0.816147
:( _> 0.115773
I am happy _> 0.518581
This course is great _> 0.515933
I do not expect so much from my soccer team _> 0.494500
It was a good book _> 0.513107
I am not sure about the text _> 0.501005


/var/folders/42/fjzlf7bn6573xz4q1rvt4rpm0000gn/T/ipykernel_7059/2009072022.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print( '%s _> %f' % (tweet, func_previsao(tweet, freqs, pesos)))


## Avaliação do Modelo

Vamos agora avaliar a performance do modelo.

Vamos criar uma função para testar o modelo que executará as seguintes tarefas:

* Recebe os dados de teste e os pesos do modelo treinado e calcula a precisão do modelo.
* Usa a função `func_previsao` para fazer previsões sobre cada tweet no conjunto de teste.
* Se a previsão for > 0,5, definimos a classificação do modelo 'y_hat' como 1, caso contrário, definimos a classificação do modelo `y_pred` como 0.
* Uma previsão é precisa quando o `y_pred` é igual ao `y_teste`. 

In [71]:
# Função para testar o modelo
def func_testa_modelo(test_x, test_y, freqs, theta):
    
    # Lista para as previsões
    y_hat = []
    
    # Loop pelos dados
    for tweet in test_x:
        
        # Faz a previsão
        y_pred = func_previsao(tweet, freqs, theta)
        
        # Cutoff
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)
            
    accuracy = (y_hat==np.squeeze(test_y)).sum() / len(test_x)
    
    return accuracy

In [72]:
acuracia = func_testa_modelo(dados_teste_x, y_teste, freqs, pesos)

In [73]:
print(f'Acurácia do Modelo = {acuracia:.4f}')

Acurácia do Modelo = 0.4980


# Deploy do Modelo Treinado e Uso com Novos Dados

In [74]:
meu_tweet_l = 'This is a great course. I am learning a lot!'

In [75]:
print(limpa_processa_tweet(meu_tweet_l))

['great', 'cours', 'learn', 'lot']


In [76]:
# Previsão
y_hat = func_previsao(meu_tweet_l, freqs, pesos)
print(y_hat)

[[0.5242686]]


In [77]:
# Cutoff
if y_hat > 0.5:
    print('O Tweet tem sentimento positivo!')
else:
    print('O Tweet tem sentimento negativo!')

O Tweet tem sentimento positivo!


In [78]:
# Cria um tweet
meu_tweet_2 = 'Today the weather is terrible :( !'

In [79]:
print(limpa_processa_tweet(meu_tweet_2))

['today', 'weather', 'terribl', ':(']


In [80]:
# Previsão
y_hat = func_previsao(meu_tweet_2, freqs, pesos)
print(y_hat)

[[0.11389124]]


In [81]:
# Cutoff
if y_hat > 0.5:
    print('O Tweet tem sentimento positivo!')
else:
    print('O Tweet tem sentimento negativo!')

O Tweet tem sentimento negativo!
